In [2]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
!pip install -q /lib/wheels/tensorflow-2.9.1-cp38-cp38-linux_x86_64.whl
!pip install -q tensorflow-addons==0.18.0
!pip install -q tensorflow-probability==0.17.0
!pip install -q opencv-python-headless
!pip install -q seaborn


!pip install -qU wandb
!pip install -qU scikit-learn

ERROR: tensorflow-2.9.1-cp38-cp38-linux_x86_64.whl is not a supported wheel on this platform.


In [12]:
# CHANGED FOR TPU 1VM:
# Apparently you may use different seed values at each stage
seed_value= 0

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)


# For working on GPUs from "TensorFlow Determinism"
os.environ["TF_DETERMINISTIC_OPS"] = "1"


# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
#                   random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
#                   np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)

# for later versions:
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=4, inter_op_parallelism_threads=4)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)
# /content/drive/MyDrive/QMISG_Prostate_Dataset&Codes/Prostate_Dataset/ProstateX Challenge/Preprocessed_Data/2D_MultiSeq_SingleSlice_Lesion/QISO/QISO_CropFixed_32px
# Imports
from sklearn.model_selection import train_test_split
import pandas as pd
import glob
import random
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
# Get the current date and time
current_datetime = datetime.now().strftime('%Y-%m-%d %H:%M')  # Format: YYYY-MM-DD HH:MM
import numpy as np
import matplotlib.pyplot as plt
import glob, os
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.optimizers import SGD, RMSprop
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras.layers import concatenate


# Imports

import keras
# from keras.utils import np_utils
keras.utils.set_random_seed(seed_value)
from tensorflow.keras.layers import (
  BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense , Input
)
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras import callbacks
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
#                   import seaborn as sns
from tensorflow.keras.applications import ResNet50V2
from keras import layers

import keras.backend as K
import tensorflow as tf

import os
import tensorflow_datasets as tfds
# import tensorflow_addons as tfa
# import tensorflow_probability as tfp
# Detect hardware, return appropriate distribution strategy
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect(tpu="local") # "local" for 1VM TPU
#     strategy = tf.distribute.TPUStrategy(tpu)
# except tf.errors.NotFoundError:
#     strategy = tf.distribute.MirroredStrategy()

# print("REPLICAS: ", strategy.num_replicas_in_sync)


In [ ]:
RUN_BY = "Moodi"
COMMENT = "Resnetcombined"
Fixed_CropSize = 64
EPOCHS = 20
BATCHSIZE = 32
RESAMPLING = "ISO_zNorm_v1"
RESIZE = True
SMOTE_STATE = True
height=Fixed_CropSize
width=Fixed_CropSize
channelss = [0, 1, 2, 3, 4]
# channels = [2,3,4] #0:T2_1:ADC_2:BVAL_3:DWI_4:Ktrans
num_channels= 3

In [ ]:
# Define directory paths
output_dir = f"/kaggle/working/{RUN_BY}/Figures"  # Modify the path as needed

# Check if the directory exists, and if not, create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
# import requests

# url = "https://api.isic-archive.com/api/v2/collections/"
# headers = {
#     "accept": "application/json",
#     "X-CSRFToken": "Kk1COEFP2gEFvnknSU4F4ZOyFlXhyTKHOZgpFPi1pFEnAH6IRFTLpXEA5gwL9U2Q",
# }

# response = requests.get(url, headers=headers)

# # You can then work with the response, e.g., check the status code or content.
# print(response.status_code)
# print(response.text)



In [ ]:
# /kaggle/input/siim-isic-melanoma-classification

In [ ]:
import pandas as pd
import glob
import numpy as np

main_directory = '/kaggle/input/siim-isic-melanoma-classification/jpeg'
image_directory = main_directory + '/train'

# Load the CSV file
csv_file = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/train.csv')

# Define a function to find the image paths and labels
def find_images_and_labels(csv_file, image_directory):
    image_paths, labels = [], []
    count = 0
    for idx, row in csv_file.iterrows():
        image_id = row['image_name']
        label = row['target']
        count+=1
#         print(image_id, label)
# /kaggle/input/siim-isic-melanoma-classification/jpeg/train/ISIC_0015719.jpg
        # Search for image files in the specified directory
        image_files = glob.glob(image_directory + f'/{image_id}.jpg')
#         print(image_files)
        # Add the found paths and labels to the lists
        if len(image_files) > 0:
            image_paths.extend(image_files)
            labels.extend([label] * len(image_files))

#         if count == 500:
#             break
    return image_paths, labels

# Find image paths and labels for the train set
X_train_path, Y_train = find_images_and_labels(csv_file, image_directory)


In [ ]:
# csv_file["image_name"].loc("ISIC_2637011")

In [ ]:
# X_train_path[0], X_train_path[1]

In [ ]:
# for path in X_train_path:
#     csv_file.loc[path[-16:-4]].add["x_col"]=path
#     csv_file['image_name'] = csv_file

In [ ]:
# csv_file_test = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/test.csv')
# image_directory_test = main_directory + '/test'
# X_test, Y_test = find_images_and_labels(csv_file_test, image_directory)

In [ ]:
# import tensorflow as tf
# import numpy as np
# import glob

# def load_and_preprocess_images(image_paths, target_size=(224, 224)):
#     images = []
#     count = 0
#     for image_path in image_paths:
#         # Load the image using TensorFlow's load_img
#         image = tf.keras.utils.load_img(image_path, target_size=target_size)
#         # Convert the loaded image to a NumPy array
#         input_arr = tf.keras.utils.img_to_array(image)
#         # Normalize pixel values to the range [0, 1]
#         input_arr = input_arr / 255.0
#         images.append(input_arr)
#         count+=1
#         if count%2000==0:
#             print(count)
#     return np.array(images)

# images = load_and_preprocess_images(X_train_path)



# # images = load_and_preprocess_images(X_train_path[:50])

In [ ]:
import tensorflow as tf
import numpy as np
import glob
from concurrent.futures import ThreadPoolExecutor

def load_and_preprocess_image(image_path, target_size=(224, 224)):
    # Load the image using TensorFlow's load_img
    image = tf.keras.utils.load_img(image_path, target_size=target_size)
    # Convert the loaded image to a NumPy array
    input_arr = tf.keras.utils.img_to_array(image)
    # Normalize pixel values to the range [0, 1]
    input_arr = input_arr / 255.0
    return input_arr

def load_and_preprocess_images(image_paths, target_size=(224, 224), num_threads=4):
    images = []

    # Create a ThreadPoolExecutor with the specified number of threads
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        count = 0
        for image in executor.map(load_and_preprocess_image, image_paths):
            images.append(image)
            count += 1
            if count % 2000 == 0:
                print("Loaded", count, "images")

    return np.array(images)

# List of image file paths
# X_train_path = glob.glob("/path/to/your/images/*.jpg")

# Example usage:
images = load_and_preprocess_images(X_train_path)



In [ ]:
# import tensorflow as tf
# import numpy as np
# import glob
# from concurrent.futures import ThreadPoolExecutor
# import gc  # Import the garbage collection module

# def load_and_preprocess_image(image_path, target_size=(224, 224)):
#     # Load the image using TensorFlow's load_img
#     image = tf.keras.utils.load_img(image_path, target_size=target_size)
#     # Convert the loaded image to a NumPy array
#     input_arr = tf.keras.utils.img_to_array(image)
#     # Normalize pixel values to the range [0, 1]
#     input_arr = input_arr / 255.0
#     return input_arr

# def load_and_preprocess_images(image_paths, target_size=(224, 224), num_threads=4, batch_size=2000):
#     images = []

#     # Create a ThreadPoolExecutor with the specified number of threads
#     with ThreadPoolExecutor(max_workers=num_threads) as executor:
#         count = 0
#         for image in executor.map(load_and_preprocess_image, image_paths):
#             images.append(image)
#             count += 1
#             if count % batch_size == 0:
#                 print("Loaded", count, "images")
#                 # Explicitly release memory
#                 del images
#                 images = []
#                 gc.collect()

#     return np.array(images)

# # List of image file paths
# # X_train_path = glob.glob("/path/to/your/images/*.jpg")

# # Example usage:
# images = load_and_preprocess_images(X_train_path)


In [ ]:
len(images),len(Y_train)

In [ ]:
# Y_train = Y_train

In [ ]:

# Convert the lists to numpy arrays
# X_train = np.array(X_train)
# Y_train = np.array(Y_train[])

# # Check the shape
# X_train.shape, Y_train.shape


In [ ]:
# Assuming you have Y_train_split, Y_valid_split, and Y_test_split as your labels

# Define a dictionary to map benign and malignant to 0 and 1
# label_mapping = {"benign": 0, "malignant": 1}

# # Update labels for the training set
# Y_train = [label_mapping[label] for label in Y_train]



In [ ]:
# Convert the list Y_train to a NumPy array
Y_train = np.array(Y_train)
Y_train

In [ ]:
# import numpy as np

# # Assuming you have X_train and Y_train as your training data and labels

# # Define the ratios
# train_ratio = 0.8
# valid_ratio = 0.1
# test_ratio = 0.1

# # Calculate the number of samples for each set
# num_samples = len(images)
# num_train = int(num_samples * train_ratio)
# num_valid = int(num_samples * valid_ratio)

# # Shuffle the data
# indices = np.random.permutation(num_samples)
# X_train_shuffled = images[indices]
# Y_train_shuffled = Y_train[indices]

# # Split the data
# X_train_split = X_train_shuffled[:num_train]
# Y_train_split = Y_train_shuffled[:num_train]

# X_valid_split = X_train_shuffled[num_train:num_train + num_valid]
# Y_valid_split = Y_train_shuffled[num_train:num_train + num_valid]

# X_test_split = X_train_shuffled[num_train + num_valid:]
# Y_test_split = Y_train_shuffled[num_train + num_valid:]


In [ ]:
!ls /content/drive/MyDrive/ISIC/

In [ ]:
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Define the directory and parameters
directory = '/content/drive/MyDrive/ISIC/'
batch_size = 32
image_size = (224, 224)

# Get the class names from the directory
class_names = sorted(os.listdir(directory))

# Load the dataset using tf.keras.preprocessing.image_dataset_from_directory
ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',  # Change this to 'binary' if you specifically want binary labels
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=42,
)




In [ ]:
#  Take the first 500 files
ds_first_500 = ds.take(50 // batch_size)

# Extract file paths and labels
file_paths = []
labels = []
for images, labels_batch in ds_first_500:
    file_paths.extend(images.numpy())
    labels.extend(labels_batch.numpy())

# Split the data into train, validation, and test sets with an 80-10-10 ratio
train_paths, test_paths, train_labels, test_labels = train_test_split(
    file_paths, labels, test_size=0.2, random_state=42, stratify=labels
)

valid_paths, test_paths, valid_labels, test_labels = train_test_split(
    test_paths, test_labels, test_size=0.5, random_state=42, stratify=test_labels
)

# Create TensorFlow Datasets for train, validation, and test sets
ds_train = tf.data.Dataset.from_tensor_slices((train_paths, train_labels)).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
ds_valid = tf.data.Dataset.from_tensor_slices((valid_paths, valid_labels)).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
ds_test = tf.data.Dataset.from_tensor_slices((test_paths, test_labels)).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Print the number of examples in each set
print("Number of examples in training set:", len(train_paths))
print("Number of examples in validation set:", len(valid_paths))
print("Number of examples in test set:", len(test_paths))

In [8]:
seed_train_validation = 42 # Must be same for train_ds and val_ds
shuffle_value = True
validation_split = 0.3
# Define the directory and parameters
directory = '/content/drive/MyDrive/ISIC/'
batch_size = 32
image_size = (224, 224)
ds_train = tf.keras.utils.image_dataset_from_directory(
directory =directory,
image_size = (224, 224),
validation_split = validation_split,
subset = "training",
seed = seed_train_validation,
color_mode = 'rgb',
shuffle = shuffle_value)

ds_val = tf.keras.utils.image_dataset_from_directory(
directory =directory,
image_size = (224, 224),
validation_split = validation_split,
subset = "validation",
seed = seed_train_validation,
color_mode = 'rgb',
shuffle = False
)
val_batches = tf.data.experimental.cardinality(ds_val)
ds_test = ds_val.take((2*val_batches) // 3)
ds_valid = ds_val.skip((2*val_batches) // 3)

Found 39015 files belonging to 2 classes.
Using 27311 files for training.
Found 39015 files belonging to 2 classes.
Using 11704 files for validation.


In [9]:
len(ds_val)*32, len(ds_test)*32

(11712, 7808)

In [ ]:
# import numpy as np
# from sklearn.model_selection import train_test_split

# # Assuming you have 'images' and 'Y_train' as your training data and labels
# # Let's assume that 'Y_train' contains binary class labels (0 and 1)

# # Define the ratios
# train_ratio = 0.8
# valid_ratio = 0.1
# test_ratio = 0.1

# # Find the indices for class 0 and class 1 samples
# class_0_indices = np.where(Y_train == 0)[0]
# class_1_indices = np.where(Y_train == 1)[0]

# # Split the data for class 0 into train, validation, and test sets
# class_0_train_indices, class_0_remaining_indices = train_test_split(class_0_indices, test_size=1 - train_ratio, random_state=42)
# class_0_valid_indices, class_0_test_indices = train_test_split(class_0_remaining_indices, test_size=test_ratio / (valid_ratio + test_ratio), random_state=42)

# # Split the data for class 1 into train, validation, and test sets
# class_1_train_indices, class_1_remaining_indices = train_test_split(class_1_indices, test_size=1 - train_ratio, random_state=42)
# class_1_valid_indices, class_1_test_indices = train_test_split(class_1_remaining_indices, test_size=test_ratio / (valid_ratio + test_ratio), random_state=42)

# # Combine the class 0 and class 1 indices for each set
# train_indices = np.concatenate([class_0_train_indices, class_1_train_indices])
# valid_indices = np.concatenate([class_0_valid_indices, class_1_valid_indices])
# test_indices = np.concatenate([class_0_test_indices, class_1_test_indices])

# # Split your data into train, validation, and test sets using the calculated indices
# X_train, X_valid, X_test = images[train_indices], images[valid_indices], images[test_indices]
# Y_train, Y_valid, Y_test = Y_train[train_indices], Y_train[valid_indices], Y_train[test_indices]


In [ ]:
# len(X_train), len(X_valid), len(X_test),Y_train.shape, Y_valid.shape, Y_test.shape

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # Define data generators
# datagen = ImageDataGenerator()  # You can add more data augmentation parameters here

# # Train Dataloader
# train_data_generator = datagen.flow(X_train_split, Y_train_split, batch_size=BATCHSIZE, shuffle=True)

# # Validation Dataloader
# valid_data_generator = datagen.flow(X_valid_split, Y_valid_split, batch_size=BATCHSIZE, shuffle=False)

# # Test Dataloader
# test_data_generator = datagen.flow(X_test_split, Y_test_split, batch_size=BATCHSIZE, shuffle=False)


In [ ]:
import matplotlib.pyplot as plt

# Assuming ds_train contains images and labels
num_images_to_plot = 5

for images, labels in ds_train.take(num_images_to_plot):
    for i in range(len(images)):
        plt.figure(figsize=(4, 4))
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f'Label: {labels[i].numpy()}')
        plt.axis('off')

plt.show()


In [15]:
model_kwargs = {"image_size":224,"input_channels": 3, "filters":256, "depth":4, "kernel_size":5, "patch_size":32, "num_classes":1}

def activation_block(x):
  x = layers.Activation("gelu")(x)
  return layers.BatchNormalization()(x)


def conv_stem(x, filters: int, patch_size: int):
  x = layers.Conv2D(filters, kernel_size=patch_size, strides=patch_size)(x)
  return activation_block(x)


def conv_mixer_block(x, filters: int, kernel_size: int):
  # Depthwise convolution.
  x0 = x
  x = layers.DepthwiseConv2D(kernel_size=kernel_size, padding="same")(x)
  x = layers.Add()([activation_block(x), x0])  # Residual.

  # Pointwise convolution.
  x = layers.Conv2D(filters, kernel_size=1)(x)
  x = activation_block(x)

  return x


def get_conv_mixer_256_8(
  image_size=224, input_channels= 3, filters=256, depth=4, kernel_size=5, patch_size=32, num_classes=1
):
  """ConvMixer-256/8: https://openreview.net/pdf?id=TVHS5Y4dNvM.
  The hyperparameter values are taken from the paper.
  """
  inputs = keras.Input((image_size, image_size, input_channels))
  # x = layers.Rescaling(scale=1.0 / 255)(inputs)

  # Extract patch embeddings.
  x = conv_stem(inputs, filters, patch_size)

  # ConvMixer blocks.
  for _ in range(depth):
      x = conv_mixer_block(x, filters, kernel_size)

  # Classification block.
  x = layers.GlobalAvgPool2D()(x)
  outputs = layers.Dense(num_classes, activation="sigmoid")(x)

  return keras.Model(inputs, outputs)

custom_vgg_model = get_conv_mixer_256_8(**model_kwargs)
checkpoint_filepath = f'/content/mydrive/melanomaweights.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
  filepath=checkpoint_filepath,
  save_weights_only=False,
  verbose=1,
  monitor='val_binary_accuracy',
  mode='max',
  save_best_only=True)
import math
from keras.callbacks import Callback
from keras import backend as K


class CosineAnnealingScheduler(Callback):
  """Cosine annealing scheduler.
  """

  def __init__(self, T_max, eta_max, eta_min=0, verbose=1):
      super(CosineAnnealingScheduler, self).__init__()
      self.T_max = T_max
      self.eta_max = eta_max
      self.eta_min = eta_min
      self.verbose = verbose

  def on_epoch_begin(self, epoch, logs=None):
      if not hasattr(self.model.optimizer, 'lr'):
          raise ValueError('Optimizer must have a "lr" attribute.')
      lr = self.eta_min + (self.eta_max - self.eta_min) * (1 + math.cos(math.pi * epoch / self.T_max)) / 2
      K.set_value(self.model.optimizer.lr, lr)
      if self.verbose > 0:
          print('\nEpoch %05d: CosineAnnealingScheduler setting learning '
                'rate to %s.' % (epoch + 1, lr))

  def on_epoch_end(self, epoch, logs=None):
      logs = logs or {}
      logs['lr'] = K.get_value(self.model.optimizer.lr)







# lr_decayed_fn = (
#   tf.keras.optimizers.schedules.CosineDecayRestarts(
#       initial_learning_rate=1e-4,
#       first_decay_steps = 1000))
# sgd=SGD(learning_rate=1e-3,decay=1e-6,momentum=0.9,nesterov=True)

sgd=tf.keras.optimizers.experimental.AdamW(
  learning_rate=0.001,
  weight_decay=0.004,
  beta_1=0.9,
  beta_2=0.999,
  epsilon=1e-07,
  amsgrad=False,
  clipnorm=None,
  clipvalue=None,
  global_clipnorm=None,
  use_ema=False,
  ema_momentum=0.99,
  ema_overwrite_frequency=None,
  jit_compile=True,
  name="AdamW",

)
CA_Scheduler= CosineAnnealingScheduler(T_max=len(ds_train), eta_max=1e-2, eta_min=1e-5)
#                   with tpu_strategy.scope():

#                   from tensorflow.keras.applications.resnet50 import ResNet50
#                   from tensorflow.keras.models import Model
# #                   with tpu_strategy.scope():
#                 # Create the ResNet50 model
#                   pre_trained_model = ResNet50(include_top=False, weights=None, pooling='avg')

#                     # Load the weights from the file
#                     # weights_path = '/content/drive/MyDrive/QMISG_Prostate_Dataset&Codes/Prostate_Dataset/ProstateX Challenge/medicalimagenet/RadImageNet-ResNet50_notop.h5'
#                     # pre_trained_model.load_weights(weights_path)

#                     # Add a fully connected layer with sigmoid activation for a single class
#                   num_classes = 1
#                   x = pre_trained_model.output
#                   x = Dense(256, activation='relu')(x)
#                   predictions = Dense(num_classes, activation='sigmoid')(x)
#                   from tensorflow.keras import layers
#                   from tensorflow import keras

#                   def activation_block(x):
#                         x = layers.Activation("gelu")(x)
#                         return layers.BatchNormalization()(x)

#                   def residual_block(x, filters: int, kernel_size: int):
#                         # Convolution path.
#                         shortcut = layers.Conv2D(filters, kernel_size=1, strides=1)(x)

#                         # Residual path.
#                         x = layers.Conv2D(filters, kernel_size=kernel_size, strides=1, padding="same")(x)
#                         x = activation_block(x)
#                         x = layers.Conv2D(filters, kernel_size=kernel_size, strides=1, padding="same")(x)
#                         x = layers.BatchNormalization()(x)

#                         # Merge paths.
#                         x = layers.Add()([shortcut, x])

#                         return x

#                   def conv_mixer_block(x, filters: int, kernel_size: int):
#                         # Depthwise convolution.
#                         x0 = x
#                         x = layers.DepthwiseConv2D(kernel_size=kernel_size, padding="same")(x)
#                         x = layers.Add()([activation_block(x), x0])  # Residual.

#                         # Pointwise convolution.
#                         x = layers.Conv2D(filters, kernel_size=1)(x)
#                         x = activation_block(x)

#                         return x

#                   def get_conv_mixer_256_8(
#                         image_size=224, input_channels=3, filters=256, depth=8, kernel_size=5, patch_size=16, num_classes=1
#                     ):
#                         inputs = keras.Input((image_size, image_size, input_channels))
#                         print(inputs.shape)

#                         # First ResNet block
#                         x = layers.Conv2D(filters, kernel_size=7, strides=2)(inputs)
#                         x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

#                         # ConvMixer blocks.
#                         for _ in range(depth):
#                             x = conv_mixer_block(x, filters, kernel_size)

#                         # Classification block.
#                         x = layers.GlobalAvgPool2D()(x)
#                         outputs = layers.Dense(num_classes, activation="sigmoid")(x)

#                         return keras.Model(inputs, outputs)

#                   model_kwargs = {
#                         "image_size": 224,
#                         "input_channels": 3,
#                         "filters": 256,
#                         "depth": 4,
#                         "kernel_size": 5,
#                         "patch_size": 32,
#                         "num_classes": 1,
#                     }
#                   from tensorflow.keras import layers, Model

#                     # Define the ResNet block with skip connection
#                   def resnet_block(x, filters, kernel_size):
#                         shortcut = x
#                         x = layers.Conv2D(filters, kernel_size, padding='same')(x)
#                         x = layers.BatchNormalization()(x)
#                         x = layers.Activation('relu')(x)
#                         x = layers.Conv2D(filters, kernel_size, padding='same')(x)
#                         x = layers.BatchNormalization()(x)
#                         x = layers.Add()([x, shortcut])  # Skip connection
#                         x = layers.Activation('relu')(x)
#                         return x

#                     # Define the ConvMixer block without skip connection
#                   def conv_mixer_block(x, filters, kernel_size):
#                         # Depthwise convolution.
#                         x0 = x
#                         x = layers.DepthwiseConv2D(kernel_size=kernel_size, padding="same")(x)
#                         x = layers.Add()([activation_block(x), x0])  # Residual.

#                         # Pointwise convolution.
#                         x = layers.Conv2D(filters, kernel_size=1)(x)
#                         x = activation_block(x)

#                         return x

#                     # Function to build the ConvMixer model with the ResNet-style block
#                   def get_conv_mixer_with_resnet_block(
#                         image_size=224, input_channels=3, filters=256, depth=8, kernel_size=5, patch_size=16, num_classes=1
#                     ):
#                         inputs = keras.Input(shape=(image_size, image_size, input_channels))
#                         x = layers.Conv2D(filters, patch_size, patch_size, padding='same')(inputs)

#                         # Add the ResNet-style block
#                         x = resnet_block(x, filters, 3)  # You can customize the kernel size as needed

#                         # ConvMixer blocks.
#                         for _ in range(depth):
#                             x = conv_mixer_block(x, filters, kernel_size)

#                         # Classification block.
#                         x = layers.GlobalAveragePooling2D()(x)
#                         outputs = layers.Dense(num_classes, activation="sigmoid")(x)

#                         model = Model(inputs, outputs)
#                         return model
#                   import tensorflow as tf
#                   from tensorflow.keras.applications import ResNet50

#                             # Define the activation block
#                   def activation_block(x):
#                                 x = tf.keras.layers.Activation("gelu")(x)
#                                 x = tf.keras.layers.BatchNormalization()(x)
#                                 return x

#                             # Define the ConvMixer architecture with specified parameters
#                   def get_conv_mixer(filters=256, depth=8, kernel_size=5, patch_size=16, num_classes=1):
#                                 # Add ResNet-50 as the first layer
#                                 resnet_model = ResNet50(include_top=False, weights=None, input_shape=(224, 224, 3))
#                                 resnet_output = resnet_model.output

#                                 # Define the conv_stem function
#                                 def conv_stem(x, filters: int, patch_size: int):
#                                     x = tf.keras.layers.Conv2D(filters, kernel_size=patch_size, strides=patch_size)(x)
#                                     x = activation_block(x)
#                                     return x

#                                 # Extract patch embeddings.
#                                 x = conv_stem(resnet_output, filters, patch_size)

#                                 # ConvMixer blocks.
#                                 for _ in range(depth):
#                                     x0 = x
#                                     x = tf.keras.layers.DepthwiseConv2D(kernel_size=kernel_size, padding="same")(x)
#                                     x = tf.keras.layers.Add()([activation_block(x), x0])
#                                     x = tf.keras.layers.Conv2D(filters, kernel_size=1)(x)
#                                     x = activation_block(x)

#                                 # Classification block.
#                                 x = tf.keras.layers.GlobalAveragePooling2D()(x)
#                                 outputs = tf.keras.layers.Dense(num_classes, activation="sigmoid")(x)

#                                 # Create a new model that includes ResNet-50 and the ConvMixer architecture
#                                 combined_model = tf.keras.Model(inputs=resnet_model.input, outputs=outputs)

#                                 return combined_model

#                             # Define the parameters for the ConvMixer
#                   model_kwargs = {"filters": 256, "depth": 4, "kernel_size": 5, "patch_size": 32, "num_classes": 1}

#                             # Create the ConvMixer model with the specified parameters
#                   custom_vgg_model = get_conv_mixer(**model_kwargs)

        # Now you can use 'custom_conv_mixer_model' for your task

# Example usage
#                   model_kwargs = {
#                         "image_size": 224,
#                         "input_channels": 3,
#                         "filters": 256,
#                         "depth": 4,
#                         "kernel_size": 5,
#                         "patch_size": 32,
#                         "num_classes": 1,
#                     }

#                   custom_vgg_model = get_conv_mixer_with_resnet_block(**model_kwargs)


#                   custom_vgg_model = get_conv_mixer_256_8(**model_kwargs)

#                   custom_vgg_model = Model(inputs=pre_trained_model.input, outputs=predictions)

custom_vgg_model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0),
        optimizer=sgd,
        metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC(name='auc')])
# **Training**
# history = custom_vgg_model.fit([X_Train_Norm, Train_zone],Y_Train, epochs=60, batch_size=16,
#                                validation_data=([X_Valid_Norm, Valid_zone], Y_Valid))
# history = custom_vgg_model.fit(datagen.flow(X_Train_Norm,Y_Train, batch_size=64),
#                                validation_data=(X_Valid_Norm, Y_Valid), class_weight=class_weight, callbacks=[ model_checkpoint_callback,CA_Scheduler], epochs=300)

# # performing data argumentation by training image generator
# dataAugmentaion = ImageDataGenerator(rotation_range = 60, zoom_range = 0.1,
# fill_mode = "nearest", shear_range = 0.20, horizontal_flip = True, vertical_flip=True,
# width_shift_range = 0.1, height_shift_range = 0.1)


# # training the model
# history = custom_vgg_model.fit(dataAugmentaion.flow(X_Train_Norm,Y_Train, batch_size=64),
#  validation_data=(X_Valid_Norm, Y_Valid), class_weight=class_weight,
#    epochs=400, callbacks=[ model_checkpoint_callback,CA_Scheduler])
# Calculate the mean and std values from your dataset
#                   mean=[0.485, 0.456, 0.406]
#                   std=[0.229, 0.224, 0.225]  # RGB mean values for ImageNet dataset
#                   # Standard deviation values for ImageNet dataset

#                   # Preprocess input images using the calculated mean and std
#                   def preprocess_with_mean_std(x):
#                       x[..., 0] -= mean[0]
#                       x[..., 1] -= mean[1]
#                       x[..., 2] -= mean[2]
#                       x[..., 0] /= std[0]
#                       x[..., 1] /= std[1]
#                       x[..., 2] /= std[2]
#                       return x
# Define the augmentation settings
augmentation_settings = {
  "rotation_range": 10,
  "width_shift_range": 0.25,
  "height_shift_range": 0.25,
  "brightness_range": None,
  "shear_range": 0.25,
  "zoom_range": 0.4,
  "channel_shift_range": 0,
  "horizontal_flip": True,
  "vertical_flip": True,
  "fill_mode": 'constant',  # Set the fill mode to "constant"
  "cval": 0.0  # Set the constant value to 0.0 for filling blank
}

dataAugmentaion = tf.keras.preprocessing.image.ImageDataGenerator(
**augmentation_settings)

# dataAugmentaion_VT = tf.keras.preprocessing.image.ImageDataGenerator(
#     preprocessing_function=preprocess_with_mean_std)

# training the model
history = custom_vgg_model.fit(ds_train,
validation_data=(ds_valid),
epochs=1, callbacks=[ model_checkpoint_callback,CA_Scheduler], shuffle=True)


# training the model
# history = custom_vgg_model.fit(
#     dataAugmentaion.flow(X_Train_Norm, Y_Train, batch_size=32),
#     validation_data=dataAugmentaion_VT.flow(X_Valid_Norm, Y_Valid),
#     epochs=300, callbacks=[model_checkpoint_callback, CA_Scheduler], shuffle=True
  # )





Epoch 00001: CosineAnnealingScheduler setting learning rate to 0.01.
854/854 [==============================] - ETA: 0s - loss: 0.3890 - binary_accuracy: 0.8232 - auc: 0.8155 
Epoch 1: val_binary_accuracy improved from -inf to 0.49179, saving model to /content/mydrive/melanomaweights.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


854/854 [==============================] - 6758s 8s/step - loss: 0.3890 - binary_accuracy: 0.8232 - auc: 0.8155 - val_loss: 0.8744 - val_binary_accuracy: 0.4918 - val_auc: 0.0000e+00 - lr: 0.0100


In [ ]:
custom_vgg_model.load_weights(checkpoint_filepath)

In [ ]:

# **Plots**
max_valAUC = max(history.history['val_auc'])

acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_ax = range(1,len(val_loss)+1)


plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(epochs_ax, acc, label='Training Accuracy')
plt.plot(epochs_ax, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(epochs_ax, loss, label='Training Loss')
plt.plot(epochs_ax, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('loss')
plt.ylim([0,2.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.savefig(f"/kaggle/working/{RUN_BY}/Figures/Loss_{current_datetime}")
plt.show()

# **Prediction after Warming up**


# Get predictions on augmented validation data
# prediction_Warmup = custom_vgg_model.predict(dataAugmentaion_VT.flow(X_Test_Norm, shuffle=False))


# # prediction_Warmup = custom_vgg_model.predict([X_Test_Norm, Test_zone]).round()
prediction_Warmup = custom_vgg_model.predict(X_test)

prediction_Warmup.shape
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import roc_curve, roc_auc_score

# Assuming Y_Test and prediction_Warmup contain the true labels and model predictions, respectively.

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(Y_test, prediction_Warmup)

# Calculate Youden's J statistic for each threshold
j_stat = tpr + (1 - fpr) - 1

# Find the threshold that maximizes Youden's J statistic
best_threshold = thresholds[np.argmax(j_stat)]

# Calculate the corresponding true positive rate (sensitivity) and false positive rate (1 - specificity)
best_tpr = tpr[np.argmax(j_stat)]
best_fpr = fpr[np.argmax(j_stat)]

# Calculate the AUC score for reference
auc_score = roc_auc_score(Y_test, prediction_Warmup)

# Plot the ROC curve with the best threshold
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {auc_score:.2f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.scatter(best_fpr, best_tpr, c='red', label=f'Best Threshold = {best_threshold:.2f}')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.savefig(f"/kaggle/working/{RUN_BY}/Figures/AUC_{current_datetime}")
plt.show()

print("Best Threshold:", best_threshold)


from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Convert predicted probabilities to binary predictions using the best threshold
binary_predictions = (prediction_Warmup >= best_threshold).astype(int)

# Convert true labels to binary format
y_true = Y_test

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_true, binary_predictions)

# Calculate sensitivity (true positive rate)
sensitivity = recall_score(y_true, binary_predictions)

# Calculate specificity (true negative rate)
specificity = recall_score(y_true, binary_predictions, pos_label=0)

# Calculate precision
precision = precision_score(y_true, binary_predictions)

# Calculate F1 score
f1 = f1_score(y_true, binary_predictions)

# Calculate accuracy
accuracy = accuracy_score(y_true, binary_predictions)

print("Confusion Matrix:")
print(conf_matrix)

print("Sensitivity (True Positive Rate):", sensitivity)
print("Specificity (True Negative Rate):", specificity)
print("Precision:", precision)
print("F1 Score:", f1)
print("Accuracy:", accuracy)

import csv
import os
import numpy as np
import json

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Define the file path for the log data
log_file_path = f'/kaggle/working/{RUN_BY}/PROSTATEX_log_data.csv'

# Check if the log file exists
if os.path.exists(log_file_path):
  # If the file exists, try to load the existing log data
  try:
      existing_log_data = []
      with open(log_file_path, 'r') as file:
          reader = csv.DictReader(file)
          for row in reader:
              existing_log_data.append(row)
  except Exception as e:
      print(f"Error loading log data: {e}")
      existing_log_data = []
else:
  # If the file doesn't exist, create an empty list for the log data
  existing_log_data = []

# Calculate additional evaluation metrics
binary_predictions = (prediction_Warmup >= best_threshold).astype(int)
y_true = Y_test_split
conf_matrix = confusion_matrix(y_true, binary_predictions)
sensitivity = recall_score(y_true, binary_predictions)
specificity = recall_score(y_true, binary_predictions, pos_label=0)
precision = precision_score(y_true, binary_predictions)
f1 = f1_score(y_true, binary_predictions)
accuracy = accuracy_score(y_true, binary_predictions)
auc = roc_auc_score(y_true, prediction_Warmup)


model_layers = [layer.name for layer in custom_vgg_model.layers]


# Define the initial log data
initial_log_data = {

  "Date of Run": current_datetime,
  "Run By": RUN_BY,
#   "Channels": channels,
  "RESAMPLING" : "Q"+str(RESAMPLING),
  "CropSize" : Fixed_CropSize,
  "Resize" : RESIZE,
  "SMOTE" : SMOTE_STATE,
  "EPOCHS" : EPOCHS,
  "BATCHSIZE" : BATCHSIZE,
  "Augmentation_settings": json.dumps(augmentation_settings),
  "Model_Config": json.dumps(model_kwargs),
  "Model_layers": model_layers,
  "Optimizer" : [CA_Scheduler.T_max, CA_Scheduler.eta_max, CA_Scheduler.eta_min],
  "Loss Type": "binary" if isinstance(custom_vgg_model.loss, tf.keras.losses.BinaryCrossentropy) else "categorical",
  "Optimizer Used": str(custom_vgg_model.optimizer),
  "Comment":COMMENT
}

# Add the initial log data, evaluation metrics, and date to the log data
evaluation_metrics = {
  "Sensitivity": sensitivity,
  "Specificity": specificity,
  "Precision": precision,
  "F1 Score": f1,
  "Accuracy": accuracy,
  "Max_Val_AUC" : max_valAUC,
  "AUC": auc,
  "Best Threshold": best_threshold,
  "Loss_fig_path": f"/kaggle/working//{RUN_BY}/Figures/Loss_{current_datetime}",
  "AUC_fig_path": f"/kaggle/working//{RUN_BY}/Figures/AUC_{current_datetime}"
}

# Combine initial log data and evaluation metrics
combined_log_data = {**initial_log_data, **evaluation_metrics}

# Append the combined log data to the existing log data
existing_log_data.append(combined_log_data)

# Get all unique field names including keys from existing_log_data
fieldnames = set().union(*(d.keys() for d in existing_log_data))

# Save the updated log data to the CSV file in Google Drive
# Define the desired order of columns
desired_column_order = [
  "Date of Run",
  "Run By",
#   "Channels",
  "RESAMPLING",
  "CropSize",
  "Resize",
  "SMOTE",
  "EPOCHS",
  "BATCHSIZE",
  "Augmentation_settings",
  "Model_Config",
  "Model_layers",
  "Optimizer",
  "Loss Type",
  "Optimizer Used",
  "Comment",
  "Sensitivity",
  "Specificity",
  "Precision",
  "F1 Score",
  "Accuracy",
  "Max_Val_AUC",
  "AUC",
  "Best Threshold",
  "Loss_fig_path",
  "AUC_fig_path"
]

# Save the updated log data to the CSV file in Google Drive
with open(log_file_path, 'w', newline='') as file:
  writer = csv.DictWriter(file, fieldnames=desired_column_order)
  writer.writeheader()
  writer.writerows(existing_log_data)

print("Log data saved successfully.")
